In [1]:
using Revise
using PauliPropagation

In [2]:
nq = 8

8

In [3]:
op = PauliString(nq, :Z, round(Int, nq/2))

PauliString(nqubits: 8, 1.0 * IIIZIIII)

In [4]:
nl = 4
topo = bricklayertopology(nq; periodic=false)
# topo = get2dtopology(4, 4)
circ = hardwareefficientcircuit(nq, nl; topology=topo)
# circ = efficientsu2circuit(nq, nl; topology=topo)
fastcirc = tofastgates(circ)
m = length(fastcirc)

124

In [5]:
using Random
Random.seed!(42)
thetas = randn(m);

In [6]:
W = Inf;                   # maximal operator weight.
min_abs_coeff = 0;          # neglect small coefficients. Only for numerical and hybrid PP.

#### Numerical Pauli Propagation
Propagates numerical coefficients.

In [7]:
opsum = PauliSum(nq, op)

PauliSum(nqubits: 8, 1 Pauli term: 
 1.0 * IIIZIIII
)

In [8]:
@time dnum = mergingbfs(fastcirc, op, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
overlapwithzero(dnum) # expectation

  0.372738 seconds (985.91 k allocations: 69.369 MiB, 3.58% gc time, 81.97% compilation time)
length(dnum) = 53247


0.21720058439757214

In [9]:
@time dnum = mergingbfs(fastcirc, opsum, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
overlapwithzero(dnum) # expectation

  0.066222 seconds (359 allocations: 3.787 MiB)
length(dnum) = 53247


0.21720058439757214

#### Hybrid Pauli Propagation
Propagates numerical coefficients, but can but truncated like the surrogate.

In [10]:
max_freq = Inf   # max frequency, i.e., max number of sines and cosines per path

Inf

In [11]:
d = Dict(op.operator => NumericPathProperties(1.0))
@time dhyb = mergingbfs!(fastcirc, d, thetas; max_weight=W, max_freq=Inf, min_abs_coeff=min_abs_coeff);
@show length(dhyb)
overlapwithzero(dhyb)

  0.194879 seconds (895.92 k allocations: 49.134 MiB, 4.05% gc time, 57.19% compilation time)
length(dhyb) = 53247


0.21720058439757214

In [12]:
dhyb # TODO: fix this

Dict{UInt16, NumericPathProperties} with 53247 entries:
  0x750d => NumericPathProperties(9.342063615803493e-7, nsins=18, ncos=9, freq=…
  0xb040 => NumericPathProperties(9.36513557295548e-6, nsins=11, ncos=5, freq=2…
  0x763e => NumericPathProperties(3.818927407510158e-6, nsins=16, ncos=8, freq=…
  0x06a7 => NumericPathProperties(-0.0001441502376292454, nsins=13, ncos=8, fre…
  0x90ec => NumericPathProperties(-1.2333560289051895e-6, nsins=16, ncos=8, fre…
  0x1e05 => NumericPathProperties(2.2079024544367792e-7, nsins=17, ncos=9, freq…
  0xec7c => NumericPathProperties(1.4947093598678283e-7, nsins=15, ncos=9, freq…
  0xea1a => NumericPathProperties(3.270505758837586e-7, nsins=19, ncos=11, freq…
  0x0d4e => NumericPathProperties(-0.0035533574412631294, nsins=11, ncos=6, fre…
  0x7084 => NumericPathProperties(-5.727900984982438e-7, nsins=19, ncos=11, fre…
  0x6bf8 => NumericPathProperties(-3.3665247394632922e-6, nsins=13, ncos=7, fre…
  0xf2e2 => NumericPathProperties(-8.123208720289147e

In [13]:
show(dhyb)

Dict{UInt16, NumericPathProperties} with 53247 entries:
  XZIIXXZX => NumericPathProperties(9.342063615803493e-7, nsins=18, ncos=9, freq=40) 
  IIIXIIZY => NumericPathProperties(9.36513557295548e-6, nsins=11, ncos=5, freq=29) 
  YZZIYXZX => NumericPathProperties(3.818927407510158e-6, nsins=16, ncos=8, freq=42) 
  ZXYYYXII => NumericPathProperties(-0.0001441502376292454, nsins=13, ncos=8, freq=31) 
  IZYZIIXY => NumericPathProperties(-1.2333560289051895e-6, nsins=16, ncos=8, freq=36) 
  XXIIYZXI => NumericPathProperties(2.2079024544367792e-7, nsins=17, ncos=9, freq=35) 
  IZZXIZYZ => NumericPathProperties(1.4947093598678283e-7, nsins=15, ncos=9, freq=37) 
  YYXIYYYZ => NumericPathProperties(3.270505758837586e-7, nsins=19, ncos=11, freq=42) 
  YZIXXZII => NumericPathProperties(-0.0035533574412631294, nsins=11, ncos=6, freq=31) 
  IXIYIIZX => NumericPathProperties(-5.727900984982438e-7, nsins=19, ncos=11, freq=39) 
  IYZZZYYX => NumericPathProperties(-3.3665247394632922e-6, nsins=13, ncos

#### Pauli Propagation Surrogate
Builds a graph that can later be evaluated.

In [14]:
d2 = operatortopathdict(op.operator)
@time dsym = mergingbfs!(circ, d2, zeros(m); max_weight=W, max_freq=max_freq);
@show length(dsym)

final_nodes = collect(pth.coeff for (obs, pth) in zerofilter(dsym));
final_eval_node = PauliGateNode(parents=final_nodes, trig_inds=zeros(Int, length(final_nodes)), signs=ones(length(final_nodes)), param_idx=1, cummulative_value=0.0);
resetnodes(final_eval_node)
resetnodes(final_eval_node)
@time eval_list = gettraceevalorder(final_eval_node, zeros(m));
length(eval_list)  # The list of all nodes. The order is such that one can savely be evaluated after the other.

  0.886231 seconds (11.42 M allocations: 615.392 MiB, 24.95% gc time, 24.97% compilation time)
length(dsym) = 53247
  0.075948 seconds (32.10 k allocations: 5.161 MiB, 46.45% compilation time)


225500

In [15]:
@time expectation(eval_list, thetas)    # This is actually not always faster than numerical propagation, but in interesting cases it is by a lot.
                                        # making this always at least as fast is work in progress. Graph traversal is hard.

  0.058188 seconds (62.00 k allocations: 4.309 MiB, 58.68% compilation time)


0.2172005843975723